In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import warnings
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN
from tensorflow.keras.utils import to_categorical
from keras.optimizers import Adam, SGD, Adagrad, RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, Callback
from sklearn.metrics import classification_report
warnings.filterwarnings('ignore', category=DeprecationWarning)

2023-05-08 18:19:50.589757: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-08 18:19:50.690769: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-08 18:19:50.693563: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-08 18:19:50.693577: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

#### Preprocessing

In [2]:
train, test = [], []
z_train, z_test = [], []
chars =  ['a', 'bA', 'dA', 'lA', 'tA']
# chars = ["a", "ai", "bA", "lA", "tA"]

def normalize(seq):
    min_x, max_x = min(el[0] for el in seq), max(el[0] for el in seq)
    min_y, max_y = min(el[1] for el in seq), max(el[1] for el in seq)
    for el in seq:
        el[0] = (el[0] - min_x)/(max_x-min_x)
        el[1] = (el[1] - min_y)/(max_y-min_y)
    return seq
def read_ip(path):
    seq = []
    with open(path, 'r') as file:
        contents = file.read()
        items = contents.split()
        for i in range(1,len(items),2):
            seq.append([float(items[i]),float(items[i+1])])
        # train.append(normalize(seq))
    return normalize(seq)

for ch in chars:
    for f in os.listdir("Group24/Handwriting_Data/" + str(ch)+"/train"):
        train.append(read_ip("Group24/Handwriting_Data/" + str(ch)+"/train/" + str(f)))
        z_train.append(str(ch))
    for f in os.listdir("Group24/Handwriting_Data/" + str(ch)+"/dev"):
        test.append(read_ip("Group24/Handwriting_Data/" + str(ch) + "/dev/" + str(f)))
        z_test.append(str(ch))

In [3]:
label_map = {'a':0, 'bA':1, 'dA':2, 'lA':3, 'tA':4}
# label_map = {"a":0, "ai":1, "bA":2, "lA":3, "tA":4}
z_train_onehot = tf.keras.utils.to_categorical([label_map[x] for x in z_train], 5)
z_test_onehot = tf.keras.utils.to_categorical([label_map[x] for x in z_test], 5)

Flattening

#### Model

In [4]:
my_tensor = tf.ragged.constant(train)
model = Sequential()
model.add(SimpleRNN(32, batch_input_shape=(None, None, 2), return_sequences=True))
model.add(SimpleRNN(32, activation="relu", return_sequences=True))
# model.add(SimpleRNN(64, activation="relu", return_sequences=True))
model.add(SimpleRNN(32, activation="relu", return_sequences=False))
model.add(Dense(32, activation="relu"))
model.add(Dense(5, activation="softmax"))
loss_fn = tf.keras.losses.CategoricalCrossentropy()

adam_optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
model.compile(loss="categorical_crossentropy", optimizer=adam_optimizer, metrics=['accuracy']) 
# Train the model
my_callbacks = [ EarlyStopping(monitor='loss', min_delta=1e-4, patience=10),    TensorBoard(log_dir=f'./logdir/Q1/test')]
model_fit = model.fit(my_tensor, z_train_onehot, batch_size=len(train), epochs=10000, verbose=1, validation_split=0.0,callbacks=my_callbacks, shuffle=False, validation_batch_size=None)

hist_metric = 'accuracy'
print(f'epochs: {len(model_fit.history[hist_metric])}, acc: {model_fit.history[hist_metric][-1]}\n')
model.save(f'models/Q1/test.tf')

test_tensor = tf.ragged.constant(test)
loss, accuracy = model.evaluate(test_tensor,z_test_onehot)

print("Test loss = ", loss)
print("Test accuracy = ", accuracy)

Epoch 1/10000


2023-05-08 18:19:52.120488: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-05-08 18:19:52.120510: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-05-08 18:19:52.120526: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (piyush-HP-Spectre-x360-Convertible-13-aw2xxx): /proc/driver/nvidia/version does not exist
2023-05-08 18:19:52.120752: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


1/1 [==============================] - 2s 2s/step - loss: 1.6109 - accuracy: 0.2222
Epoch 2/10000
1/1 [==============================] - 0s 98ms/step - loss: 1.5927 - accuracy: 0.1784
Epoch 3/10000
1/1 [==============================] - 0s 96ms/step - loss: 1.5806 - accuracy: 0.1930
Epoch 4/10000
1/1 [==============================] - 0s 97ms/step - loss: 1.5702 - accuracy: 0.2018
Epoch 5/10000
1/1 [==============================] - 0s 100ms/step - loss: 1.5612 - accuracy: 0.2281
Epoch 6/10000
1/1 [==============================] - 0s 115ms/step - loss: 1.5548 - accuracy: 0.2164
Epoch 7/10000
1/1 [==============================] - 0s 126ms/step - loss: 1.5476 - accuracy: 0.2281
Epoch 8/10000
1/1 [==============================] - 0s 121ms/step - loss: 1.5400 - accuracy: 0.2602
Epoch 9/10000
1/1 [==============================] - 0s 179ms/step - loss: 1.5322 - accuracy: 0.2719
Epoch 10/10000
1/1 [==============================] - 0s 120ms/step - loss: 1.5231 - accuracy: 0.2836
Epoch 11/